In [5]:
!pip install sentence_transformers
!pip -q install langchain tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers
!pip -q install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.4 MB/s eta 0:00:00


In [6]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader

In [7]:
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip

--2025-01-30 13:57:55--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2025-01-30 13:57:55--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2025-01-30 13:57:55 (25.7 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]

A

In [8]:
loader = DirectoryLoader('./stock_market_june_2023', glob='**/*.txt', loader_cls = TextLoader)
documents = loader.load()

In [9]:
documents

[Document(metadata={'source': 'stock_market_june_2023/stock_market.txt'}, page_content='The stock market had a strong month in May, especially for those who read our previous monthly stock recommendation. At The Markets Watch, our journalists and analysts have compiled a list of monthly stock opportunities for buy-and-hold investors and traders looking to buy great companies at an even better price.\nLooking at some of our stock suggestions from the previous month, the evidence of our team’s expertise becomes compelling. In May:\n\nPalantir soared by over 74%\nNvidia witnessed a growth of over 34%\nCrowdstrike Holdings increased by over 30%\nShopify saw a rise of over 19%\nOur insights and articles about these stocks collectively yielded a remarkable 57% return last month for our readers who took action on our recommendations.\n\nInstitutional ETFs assembled by Goldman Sachs and Vanguard rely on growth stocks to drive their price in all market conditions, and your portfolio is no diffe

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [11]:
len(texts)

49

In [12]:
texts[2]

Document(metadata={'source': 'stock_market_june_2023/stock_market.txt'}, page_content='Institutional ETFs assembled by Goldman Sachs and Vanguard rely on growth stocks to drive their price in all market conditions, and your portfolio is no different. Most of our recommendations are already a staple in these ETFs, but for investors looking to cherry-pick the best stocks, we have assembled this list for you.\n\nHere are 8 growth stocks for you to buy in June 2023 set to leverage emerging industry trends, such as AI, the lithium “gold” rush, blockbuster EV sales, and ESG.')

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import transformers

model="tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
llm = HuggingFacePipeline(pipeline=pipeline)

model_name = "intfloat/e5-large-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)

<ipython-input-16-8f75202eb7f1>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipeline)
<ipython-input-16-8f75202eb7f1>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(model_name=model_name)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [17]:
# Create the DB
persist_directory = 'db'
embedding = hf
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [18]:
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

<ipython-input-18-bfd8ce2ba3d8>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
<ipython-input-18-bfd8ce2ba3d8>:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


In [19]:
retriever = vectordb.as_retriever(search_kwargs={'k': 3})

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])


In [20]:
query = "Could you please enumerate the companies that have been highligghted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

<ipython-input-20-bf9b38ae08e2>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_response = qa_chain(query)
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Nvidia’s shares have seen a massive surge since the beginning of 2023, indicating that the market recognizes the company’s substantial potential in the AI industry. While some may consider the stock to be pricy, Nvidia has a lot of things going for it to help justify the premium price tag.

Arbor Metals Corp.
Tesla Inc.
Kiplin Metals Inc.
Microsoft
NVIDIA
Shopify
Intuitive Surgical
MercadoLibre
Growth stocks like these are ideal for investors seeking short-term gains and long-term resilience. These companies are primed to withstand future market fluctuations through innovation, market opportunities, embedding in recession-resistant sectors, and committing to diversifying their portfolios.

Future-proofing your portfolio with growth stocks isn’t just about singling out products and industries that will always be in demand. 
I